In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode

In [2]:
class ImageDataset(Dataset):
    def __init__(self, im_dir):
        df = pd.read_csv(os.path.join(im_dir, "label.csv"), sep=";")
        self.images = df["FileName"].apply(lambda im: os.path.join(im_dir, im))
        self.labels = df["Side"].map({"Left": 0.0, "Right": 1.0})
        
    def __len__(self) -> int:
        return len(self.labels)
    
    def __getitem__(self, index) -> (torch.Tensor, float):
        image = read_image(self.images[index], ImageReadMode.GRAY) / 255
        label = self.labels[index]
        return image, torch.Tensor([label])

    
batch_size, n_epochs = 32, 10

train_dataloader = DataLoader(ImageDataset("../data/train/"), batch_size=batch_size, shuffle=True)
test_dataloader  = DataLoader(ImageDataset("../data/test/"), batch_size=batch_size, shuffle=True)

In [3]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.c1 = nn.Conv2d(1, 4, kernel_size=3)
        self.p1 = nn.MaxPool2d(2)
        
        self.c2 = nn.Conv2d(4, 8, kernel_size=3)
        self.p2 = nn.MaxPool2d(2)
        self.d1 = nn.Dropout2d(0.35)
        
        self.c3 = nn.Conv2d(8, 16, kernel_size=3)
        self.p3 = nn.MaxPool2d(2)
        self.d2 = nn.Dropout2d(0.35)
        
        self.f1 = nn.Flatten()
        self.l1 = nn.Linear(45 * 50 * 16, 64)
        self.d3 = nn.Dropout()
        
        self.l2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.p1(F.relu(self.c1(x)))
        x = self.d1(self.p2(F.relu(self.c2(x))))
        x = self.d2(self.p3(F.relu(self.c3(x))))
        x = self.d3(F.relu(self.l1(self.f1(x))))
        x = torch.sigmoid(self.l2(x))
        return x

    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Classifier()
model.to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

print(model, "on", device)

Classifier(
  (c1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout2d(p=0.35, inplace=False)
  (c3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): Dropout2d(p=0.35, inplace=False)
  (f1): Flatten(start_dim=1, end_dim=-1)
  (l1): Linear(in_features=36000, out_features=64, bias=True)
  (d3): Dropout(p=0.5, inplace=False)
  (l2): Linear(in_features=64, out_features=1, bias=True)
) on cuda:0


In [4]:
def run_epoch(training, verbose=True):
    dataloader = None
    if training:
        dataloader = train_dataloader
        model.train()
    else:
        dataloader = test_dataloader
        model.eval()
    
    run_loss, correct = 0, 0
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_pred = model(x_batch.to(device)).to("cpu")
        loss = criterion(y_pred, y_batch)
        if training:
            loss.backward()
            optimizer.step()
        
        run_loss += loss.item()
        correct += ((y_pred >= 0.5) == y_batch).sum()
    if verbose:
        print("Training Loss: {:8.5f} | Accuracy: {:6.2%}".format(run_loss / len(dataloader), correct / len(dataloader.dataset)))


In [ ]:
for n in range(n_epochs):
    print("Epoch {:4d}".format(n), end=" | ")
    run_epoch(training=True)
    print("Validation", end = " | ")
    run_epoch(training=False)

Epoch    0 | Training Loss:   0.65829 | Accuracy: 68.63%
Validation | Training Loss:   0.54688 | Accuracy: 76.31%
Epoch    1 | Training Loss:   0.49293 | Accuracy: 75.44%
Validation | Training Loss:   0.21857 | Accuracy: 96.46%
Epoch    2 | Training Loss:   0.25154 | Accuracy: 89.09%
Validation | Training Loss:   0.05605 | Accuracy: 98.61%
Epoch    3 | Training Loss:   0.13836 | Accuracy: 94.25%
Validation | Training Loss:   0.04655 | Accuracy: 98.29%
Epoch    4 | Training Loss:   0.12718 | Accuracy: 95.20%
Validation | Training Loss:   0.04069 | Accuracy: 98.61%
Epoch    5 | Training Loss:   0.09878 | Accuracy: 96.70%
Validation | Training Loss:   0.03998 | Accuracy: 98.39%
Epoch    6 | Training Loss:   0.07874 | Accuracy: 97.60%
Validation | Training Loss:   0.04154 | Accuracy: 98.61%
Epoch    7 | Training Loss:   0.13438 | Accuracy: 97.00%
Validation | Training Loss:   0.05142 | Accuracy: 98.29%
Epoch    8 | Training Loss:   0.07373 | Accuracy: 97.50%
Validation | Training Loss:   0